In [8]:
import face_recognition
import cv2
import MySQLdb
import datetime
import time

# Open database connection
db = MySQLdb.connect("localhost","root","","TEST" )

# prepare a cursor object using cursor() method
cursor = db.cursor()


video_capture = cv2.VideoCapture(0)

abhilash_image = face_recognition.load_image_file("Abhilash.jpeg")
abhilash_face_encoding = face_recognition.face_encodings(abhilash_image)[0]

# alka_image = face_recognition.load_image_file("Alka.jpg")
# alka_face_encoding = face_recognition.face_encodings(alka_image)[0]

himanshu_image = face_recognition.load_image_file("Himanshu.jpeg")
himanshu_face_encoding = face_recognition.face_encodings(himanshu_image)[0]

shivanshu_image = face_recognition.load_image_file("shivanshu.jpeg")
shivanshu_face_encoding = face_recognition.face_encodings(shivanshu_image)[0]

# lucky_image = face_recognition.load_image_file("lucky.jpeg")
# lucky_face_encoding = face_recognition.face_encodings(lucky_image)[0]

known_face_encodings = [
    himanshu_face_encoding,
    abhilash_face_encoding,
    shivanshu_face_encoding
]
known_face_names = [
    "Himanshu",
    "Abhilash",
    "Shivanshu"
]

face_locations = []
face_encodings = []
face_names = []
process_this_frame = True
flag = ""
recognized = ""

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Resize frame of video to 1/4 size for faster face recognition processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_small_frame = small_frame[:, :, ::-1]

    # Only process every other frame of video to save time
    if process_this_frame:
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance=0.4)
            name = "Unknown"
            
            

            # If a match was found in known_face_encodings, just use the first one.
            if True in matches:
                
                first_match_index = matches.index(True)
                #print(first_match_index)
                name = known_face_names[first_match_index]

            face_names.append(name)
            recognized = face_names[0]
    
    process_this_frame = not process_this_frame
    
    
    if not flag == recognized and not recognized == "Unknown":
      
        ts = time.time()
        timestamp = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')

        # execute SQL query using execute() method.
        cursor.execute("INSERT INTO emp_time(Name, Time) VALUES ('"+ recognized +"', '"+ timestamp +"')")

        db.commit()

        flag = recognized


    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 0), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left - 110, bottom + 70), (right + 110, bottom), (0, 0, 0), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left, bottom + 30), font, 1.0, (255, 255, 255), 1)
#         ts = time.time()
#         timestamp = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
        cv2.putText(frame, timestamp, (left - 110, bottom + 60), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()